In [39]:
"""
Inviscid Burgers Equation: Non-Conservative Form
    Spatial: WENO-5
    Temporal: RK3
"""
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import numpy.linalg


In [ ]:
def InitialCondition(x):
    return np.sin(2*np.pi*x)

In [ ]:
def BoundaryConditionDir(u):
    u[2],u[-3] = 0,0
    u[0] = 3*u[3] - 2*u[4]
    u[1] = 2*u[3] - 1*u[4]
    u[-2] = 2*u[-3]-1*u[-4]
    u[-1] = 3*u[-3]-2*u[-4]

def BoundaryConditionPer(u):
    u[2],u[-3] = 0,0
    u[[0,1]] = u[[-4,-3]]
    u[[-2,-1]] = u[[2,3]]

In [ ]:
def WENOfromLeft(v1,v2,v3,v4,v5):
    # if the flow come from left
    # the velocity value at i+1/2
    # i-2, i-1, i, i+1, i+2
    eps = 1e-6
    dL = np.array([0.1, 0.6, 0.3])
    b = np.array([
        (13.0/12.0)*(v1-2.0*v2+v3)**2 + 0.25*(v1-4.0*v2+3.0*v3)**2,
        (13.0/12.0)*(v2-2.0*v3+v4)**2 + 0.25*(v2-v4)**2,
        (13.0/12.0)*(v3-2.0*v4+v5)**2 + 0.25*(3.0*v3-4.0*v4+v5)**2
    ])
    a = dL/(b+eps)**2
    wL = a/np.sum(a)
    uL = (
        wL[0]*(v1/3.0 - 7.0/6.0*v2 + 11.0/6.0*v3) +
        wL[1]*(-v2/6.0 + 5.0/6.0*v3 + v4/3.0) +
        wL[2]*(v3/3.0 + 5.0/6.0*v4 - v5/6.0)
    )
    return uL

def WENOfromRight(v1,v2,v3,v4,v5):
    # if the flow come from left
    # the velocity value at i+1/2
    # i-2, i-1, i, i+1, i+2
    eps = 1e-6
    dR = np.array([0.3, 0.6, 0.1])
    b = np.array([
        (13.0/12.0)*(v1-2.0*v2+v3)**2 + 0.25*(v1-4.0*v2+3.0*v3)**2,
        (13.0/12.0)*(v2-2.0*v3+v4)**2 + 0.25*(v2-v4)**2,
        (13.0/12.0)*(v3-2.0*v4+v5)**2 + 0.25*(3.0*v3-4.0*v4+v5)**2
    ])
    a = dR/(b+eps)**2
    wR = a/np.sum(a)
    uR = (
        wR[0]*(-1.0/6.0*v1 - 5.0/6.0*v2 + 1.0/3.0*v3) +
        wR[1]*(v2/3.0 + 5.0/6.0*v3 + v4/6.0) +
        wR[2]*(11.0*v3/6.0 - 7.0/6.0*v4 +1.0/3.0*v5)
    )
    return uR

In [ ]:
def RK3WENO(uOrig, dX, dT, alpha, BCfun):
    # NOTE: the journal is incorrect please have a look on
    # Proposition 3.2 in TOTAL VARIATION DIMINISHING RUNGE-KUTTA SCHEMES
    uOne = uOrig*np.nan
    uTwo = uOrig*np.nan
    uNow = uOrig*np.nan
    # First Intermediate
    uOne[3:-3] = uOrig[3:-3] + dT/dX * (
        max(uOrig[3:-3],0) * (
            WENOfromLeft(uOrig[1:-5], uOrig[2:-4], uOrig[3:-3], uOrig[4:-2], uOrig[5:-1]) -
            WENOfromLeft(uOrig[0:-6], uOrig[1:-5], uOrig[2:-4], uOrig[3:-3], uOrig[4:-2])
        ) +
        min(uOrig[3:-3],0) * (
            WENOfromRight(uOrig[2:-4], uOrig[3:-3], uOrig[4:-2], uOrig[5:-1], uOrig[6:]) -
            WENOfromRight(uOrig[0:-6], uOrig[1:-5], uOrig[2:-4], uOrig[3:-3], uOrig[4:-2])
        )
    )
    BCfun(uOne)

    # Second Intermediate
    uTwo[3:-3] = 3./4. * uOrig[3:-3] + 1./4. * uOne[3:-3] + 0.25 * dT/dX * (
        max(uOne[3:-3],0) * (
            WENOfromLeft(uOne[1:-5], uOne[2:-4], uOne[3:-3], uOne[4:-2], uOne[5:-1]) -
            WENOfromLeft(uOne[0:-6], uOne[1:-5], uOne[2:-4], uOne[3:-3], uOne[4:-2])
        ) +
        min(uOne[3:-3],0) * (
            WENOfromRight(uOne[2:-4], uOne[3:-3], uOne[4:-2], uOne[5:-1], uOne[6:]) -
            WENOfromRight(uOne[0:-6], uOne[1:-5], uOne[2:-4], uOne[3:-3], uOne[4:-2])
        )
    )
    BCfun(uTwo)

    # Final
    uNow[3:-3] = 1./3. * uOrig[3:-3] + 2./3. * uTwo[3:-3] + 2.0/3.0 * dT/dX * (
        max(uTwo[3:-3],0) * (
            WENOfromLeft(uTwo[1:-5], uTwo[2:-4], uTwo[3:-3], uTwo[4:-2], uTwo[5:-1]) -
            WENOfromLeft(uTwo[0:-6], uTwo[1:-5], uTwo[2:-4], uTwo[3:-3], uTwo[4:-2])
        ) +
        min(uTwo[3:-3],0) * (
            WENOfromRight(uTwo[2:-4], uTwo[3:-3], uTwo[4:-2], uTwo[5:-1], uTwo[6:]) -
            WENOfromRight(uTwo[0:-6], uTwo[1:-5], uTwo[2:-4], uTwo[3:-3], uTwo[4:-2])
        )
    )
    BCfun(uNow)
    return uNow

In [ ]:
xMin, xMax = (0., 1.)
numPoint = 81
xPoints = np.linspace(xMin, xMax, num=numPoint)
deltaX = xPoints[1] - xPoints[0]
xPwG = np.array([
    *(xPoints[0:2]-2*deltaX), *xPoints,*(xPoints[-2:]+2*deltaX)
])